In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/Shareddrives/si370/final_delivery/Pilot_Test/

/content/drive/Shareddrives/si370/final_delivery/Pilot_Test


LightGBM model for Industry_Tags.csv dataset created in the MVP function(located in code.ipynb)

In [ ]:
industry_train_df = pd.read_csv("Data/training_data/training_data_from_Industry_Tags.csv_file/industry_tags_training_eng_only.csv")
industry_val_df = pd.read_csv("Data/training_data/training_data_from_Industry_Tags.csv_file/industry_tags_eval_eng_only.csv")
industry_train_df.drop(["Unnamed: 0"], axis=1, inplace=True)
industry_val_df.drop(["Unnamed: 0"], axis=1, inplace=True)
train_eval_df = pd.concat([industry_train_df, industry_val_df])

## labels and tags crosswalk##
# 'Agriculture, Fishing and Forestry': 0,
# 'Education': 1,
# 'Energy and Extractives': 2,
# 'Financial Sector': 3,
# 'Health': 4,
# 'Industry, Trade and Services': 5,
# 'Information and Communications Technologies': 6,
# 'Public Administration': 7,
# 'Social Protection': 8,
# 'Transportation': 9,
# 'Water, Sanitation and Waste Management': 10

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords1 = set(stopwords.words('english'))

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

import lightgbm as lgbm
from lightgbm import LGBMClassifier

tfidf_vec = TfidfVectorizer(dtype=np.float32, sublinear_tf=True, use_idf=True, smooth_idf=True)
# Fit the whole dataset before spliting into train and eval set
data_tfidf = tfidf_vec.fit_transform(train_eval_df['text'])
X_train, X_test, y_train, y_test = train_test_split(train_eval_df['text'], train_eval_df['label'], random_state = 0, test_size=0.1, shuffle=True)

X_train_tfidf = tfidf_vec.transform(X_train)
X_test_tfidf = tfidf_vec.transform(X_test)

num_consumer_goods = len(train_eval_df[train_eval_df["label"] == 0])
num_extractives = len(train_eval_df[train_eval_df["label"] == 1])
num_financials = len(train_eval_df[train_eval_df["label"] == 2])
num_food = len(train_eval_df[train_eval_df["label"] == 3])
num_health_care = len(train_eval_df[train_eval_df["label"] == 4])
num_infrastructure = len(train_eval_df[train_eval_df["label"] == 5])
num_renewable = len(train_eval_df[train_eval_df["label"] == 6])
num_resource = len(train_eval_df[train_eval_df["label"] == 7])               
num_services = len(train_eval_df[train_eval_df["label"] == 8])                      
num_technology = len(train_eval_df[train_eval_df["label"] == 9])     
num_transportation = len(train_eval_df[train_eval_df["label"] == 10])    

model = LGBMClassifier(boosting_type='dart',
    objective='multiclass',
    learning_rate=0.05,
    num_iteration=100, 
    num_leaves=24,
    max_depth=5,
    num_class=11,
    class_weight={
        0: (num_consumer_goods/len(train_eval_df)), 
        1: (num_extractives/len(train_eval_df)),
        2: (num_financials/len(train_eval_df)),
        3: (num_food/len(train_eval_df)),
        4: (num_health_care/len(train_eval_df)),
        5: (num_infrastructure/len(train_eval_df)),
        6: (num_renewable/len(train_eval_df)),
        7: (num_resource/len(train_eval_df)),
        8: (num_services/len(train_eval_df)),
        9: (num_technology/len(train_eval_df)),
        10: (num_transportation/len(train_eval_df))
        })

model.fit(X_train_tfidf, y_train)

predicted_LGBM = model.predict(X_test_tfidf)

print(accuracy_score(y_test, predicted_LGBM))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.5416666666666666


The result was bad, so I moved on to the sasb 10-K industry tag data

In [ ]:
sasb_data = pd.read_csv("Data/training_data/sasb_full_training.csv")

In [ ]:
labels = sasb_data.Sector.unique()

label_dict = {}
for index, label in enumerate(sorted(labels)):
    label_dict[label] = index
label_dict

{'consumer goods': 0,
 'extractives & minerals processing': 1,
 'financials': 2,
 'food & beverage': 3,
 'health care': 4,
 'infrastructure': 5,
 'renewable resources & alternative energy': 6,
 'resource transformation': 7,
 'services': 8,
 'technology & communications': 9,
 'transportation': 10}

In [ ]:
sasb_data['label'] = sasb_data.Sector.replace(label_dict)

In [ ]:
sasb_data.drop(["Unnamed: 0"], axis=1, inplace=True)
sasb_data

,train_text,Sector,label
0,development distribution and manufacture of a ...,health care,4
1,the worldwide development manufacture and dist...,health care,4
2,201 847 6800 All references herein to the Comp...,health care,4
3,201 847 6800 All references herein to the Comp...,health care,4
4,201 847 6800 All references in this Form 10 K ...,health care,4
...,...,...,...
2796,along with our joint venture partners are the ...,food & beverage,3
2797,along with our joint venture partners are the ...,food & beverage,3
2798,in 2014 in Delaware without its subsidiaries 8...,extractives & minerals processing,1
2799,in 2014 in Delaware without its subsidiaries q...,extractives & minerals processing,1


I created this csv file so that Anne can load it direclty from google drive

In [ ]:
sasb_data.to_csv("sasb_train_eval_dataset.csv")

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords1 = set(stopwords.words('english'))

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

import lightgbm as lgbm
from lightgbm import LGBMClassifier

# import LGBMClassifier卻沒有用到 要用這個才對

tfidf_vec = TfidfVectorizer(dtype=np.float32, sublinear_tf=True, use_idf=True, smooth_idf=True)
#先把整份資料fit過再分別transform 向我做ner一樣
data_tfidf = tfidf_vec.fit_transform(sasb_data['train_text'])
X_train, X_test, y_train, y_test = train_test_split(sasb_data['train_text'], sasb_data['label'], random_state = 0, test_size=0.1, shuffle=True)

X_train_tfidf = tfidf_vec.transform(X_train)
X_test_tfidf = tfidf_vec.transform(X_test)

# lgbm_train = lgbm.Dataset(X_train_tfidf, y_train)
# lgbm_eval = lgbm.Dataset(X_test_tfidf, y_test, reference=lgbm_train)

num_consumer_goods = len(sasb_data[sasb_data["label"] == 0])
num_extractives = len(sasb_data[sasb_data["label"] == 1])
num_financials = len(sasb_data[sasb_data["label"] == 2])
num_food = len(sasb_data[sasb_data["label"] == 3])
num_health_care = len(sasb_data[sasb_data["label"] == 4])
num_infrastructure = len(sasb_data[sasb_data["label"] == 5])
num_renewable = len(sasb_data[sasb_data["label"] == 6])
num_resource = len(sasb_data[sasb_data["label"] == 7])               
num_services = len(sasb_data[sasb_data["label"] == 8])                      
num_technology = len(sasb_data[sasb_data["label"] == 9])     
num_transportation = len(sasb_data[sasb_data["label"] == 10])    

# I tried several ways to define the class weight argument 
# this version got the highest accuracy on the test data
model = LGBMClassifier(boosting_type='dart',
    objective='multiclass',
    learning_rate=0.05,
    num_iteration=300, 
    num_leaves=24,
    max_depth=5,
    num_class=11,
    class_weight={
        0: (num_consumer_goods/(len(sasb_data)-num_consumer_goods)),
        1: (num_extractives/(len(sasb_data)-num_extractives)),
        2: (num_financials/(len(sasb_data)-num_financials)),
        3: (num_food/(len(sasb_data)-num_food)),
        4: (num_health_care/(len(sasb_data)-num_health_care)),
        # 5: (num_infrastructure/len(sasb_data)),
        5:0.05,
        6: (num_renewable/(len(sasb_data)-num_renewable)),
        7: (num_resource/(len(sasb_data)-num_resource)),
        8: (num_services/(len(sasb_data)-num_services)),
        9: (num_technology/(len(sasb_data)-num_technology)),
        10: (num_transportation/(len(sasb_data)-num_transportation))
        })

model.fit(X_train_tfidf, y_train)

predicted_LGBM = model.predict(X_test_tfidf)

print(accuracy_score(y_test, predicted_LGBM))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.9217081850533808


The model

In [ ]:
model

LGBMClassifier(boosting_type='dart',
               class_weight={0: 0.1106264869151467, 1: 0.10275590551181102,
                             2: 0.08776699029126214, 3: 0.06018168054504164,
                             4: 0.08230293663060279, 5: 0.05,
                             6: 0.012287676183592338, 7: 0.17738545607398065,
                             8: 0.050243719535058115, 9: 0.12129703763010408,
                             10: 0.09115699259836385},
               colsample_bytree=1.0, importance_type='split',
               learning_rate=0.05, max_depth=5, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_class=11, num_iteration=300, num_leaves=24,
               objective='multiclass', random_state=None, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

Save the model to google drive

In [ ]:
model.booster_.save_model("lightgbm_sasb_model.txt")

# Load the model and use it to label 

Load the model from google drive

In [ ]:
# import lightgbm as lgbm
model = lgbm.Booster(model_file="lightgbm_sasb_model.txt")

Test with some text which the model hasn't seen

In [ ]:
moderna = "Since our founding in 2010 we have worked to build the industry's leading mRNA technology platform the infrastructure to accelerate drug discovery and early development a rapidly expanding pipeline and a world-class team Our pipeline includes development candidates for mRNA-based vaccines and therapies spanning several therapeutic areas, and we have multiple clinical trials underway with other development candidates progressing toward the clinic. In addition, we have numerous discovery programs advancing toward development."


In [ ]:
honda = """Founded in 1948 in Hamamatsu, Japan, Honda opened its first U.S. storefront in Los Angeles, CA, in 1959. What began as a nimble operation with eight industrious associates would quickly grow to astounding heights. For nearly four decades Honda has challenged to exceed government requirements – as the first automaker to meet the Clean Air Act standards in the 1970s, then demonstrating low-emissions vehicle technology that led the state of California to adopt new, more stringent emissions regulations."""

In [ ]:
transportation = """Transport infrastructure consists of the fixed installations, including roads, railways, airways, waterways, canals, and pipelines and terminals such as airports, railway stations, bus stations, warehouses, trucking terminals, refueling depots (including fueling docks and fuel stations), and seaports. Terminals may be used both for interchange of passengers and cargo and for maintenance."""

In [ ]:
airport = """An airport is an aerodrome with extended facilities, mostly for commercial air transport Airports often have facilities to park and maintain aircraft, and a control tower. An airport consists of a landing area, which comprises an aerially accessible open space including at least one operationally active surface such as a runway for a plane to take off and to land or a helipad and often includes adjacent utility buildings such as control towers, hangars and terminals."""

In [ ]:
pepsi = """Your beverage choices change by the day and sometimes by the hour. PepsiCo is there for you every sip of the way. We offer the variety of beverages you want, ingredients you trust, and package sizes that fit your lifestyle. It's all about choice and finding the match that's right for you"""

In [ ]:
amazon = """Amazon.com, Inc. is an American multinational technology company which focuses on e-commerce, cloud computing, digital streaming, and artificial intelligence. It is one of the Big Five companies in the U.S. information technology industry, along with Google, Apple, Microsoft, and Facebook. The company has been referred to as "one of the most influential economic and cultural forces in the world", as well as the world's most valuable brand"""

In [ ]:
it = """Information technology (IT) is the use of computers to store or retrieve data and information. IT is typically used within the context of business operations as opposed to personal or entertainment technologies. IT is considered to be a subset of information and communications technology (ICT). An information technology system (IT system) is generally an information system, a communications system, or, more specifically speaking, a computer system – including all hardware, software, and peripheral equipment – operated by a limited group of IT users."""

GDELT: text in url, e.g. "ebola prevention vaccine press briefing" from http://reliefweb.int/report/sierra-leone/ebola-prevention-vaccine-press-briefing

In [ ]:
gdelt1 = """ebola prevention vaccine press briefing"""

In [ ]:
gdelt2 = """ambassador general health medical disease"""

In [ ]:
gdelt3 = """Sierra Leone's officials break their own ebola emergency laws"""

In [ ]:
finance = """Finance is a term for matters regarding the management, creation, and study of money and investments. Specifically, it deals with the questions of how an individual, company or government acquires money – called capital in the context of a business – and how they spend or invest that money. Finance is then often split into the following major categories: personal finance, corporate finance, and public finance."""

GDELT: whole text from the url page

In [ ]:
sierra = """President Dr Ernest Bai Koroma has called on the people of Sierra Leone to recognize the efforts of those who are promoting the private sector as an engine of socio-economic growth. He paid special tribute to the chief executive officer and founder of Union Trust Bank (UTB) James Sanpha Koroma for his initiative, resilience and hard work in establishing the UTB 20 years ago, and acknowledged his consistency in the banking industry since then. He made this declaration during the opening ceremony of the newly constructed UTB branch and training center in Mile 91, Yoni chiefdom, Tonkolili district on Friday 20 February, 2015."""

In [ ]:
test_df= pd.DataFrame({"text":[moderna, honda, transportation, airport, pepsi, amazon, it, gdelt1, gdelt2, gdelt3, finance, sierra]})

In [ ]:
test_df["text"]

0     Since our founding in 2010 we have worked to b...
1     Founded in 1948 in Hamamatsu, Japan, Honda ope...
2     Transport infrastructure consists of the fixed...
3     An airport is an aerodrome with extended facil...
4     Your beverage choices change by the day and so...
5     Amazon.com, Inc. is an American multinational ...
6     Information technology (IT) is the use of comp...
7               ebola prevention vaccine press briefing
8             ambassador general health medical disease
9     Sierra Leone's officials break their own ebola...
10    Finance is a term for matters regarding the ma...
11    President Dr Ernest Bai Koroma has called on t...
Name: text, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
sasb_data = pd.read_csv("sasb_train_eval_dataset.csv")
tfidf_vec = TfidfVectorizer(dtype=np.float32, sublinear_tf=True, use_idf=True, smooth_idf=True)
data_tfidf = tfidf_vec.fit_transform(sasb_data['train_text'])
test = tfidf_vec.transform(test_df["text"])
pred = model.predict(test)
max = np.argmax(pred, axis=1)
max

array([ 4, 10,  7,  5,  3,  0,  9,  7,  4,  7,  7,  2])

Since there is no gold tag for the test data, I evaluated the result based on my understanding of those test text. I think the model got 9 out of the 11 test data correctly.